In [1]:
import numpy as np
import cv2
import pandas as pd
import os
import pickle

# Face Detection Model
- Load the models

In [2]:
face_detection_model = './models/res10_300x300_ssd_iter_140000.caffemodel'
face_detection_proto = './models/deploy.prototxt.txt'
face_descriptor = './models/openface.nn4.small2.v1.t7'
# load models using cv2 dnn
detector_model = cv2.dnn.readNetFromCaffe(face_detection_proto,face_detection_model)
descriptor_model = cv2.dnn.readNetFromTorch(face_descriptor)

In [3]:
!dir MCET_TEACHERS

 Volume in drive D has no label.
 Volume Serial Number is 9080-4D15

 Directory of D:\final year project\4_MachineLearning_jupyter_notebook_models_test_data\MCET_TEACHERS

25-11-2022  17:28    <DIR>          .
01-01-2023  01:26    <DIR>          ..
01-01-2023  01:26    <DIR>          Amit Sir
01-01-2023  01:26    <DIR>          Aninda Bakshi Sir
01-01-2023  01:26    <DIR>          Tanmoy Bera Sir
               0 File(s)              0 bytes
               5 Dir(s)  200,906,944,512 bytes free


In [4]:
def helper(image_path):
    img = cv2.imread(image_path)
    # step-1: face detection
    image = img.copy()
    h,w = image.shape[:2]
    img_blob = cv2.dnn.blobFromImage(image,1,(300,300),(104,177,123),swapRB=False,crop=False)
    # set the input
    detector_model.setInput(img_blob)
    detections = detector_model.forward()

    if len(detections) > 0:
        i = np.argmax(detections[0,0,:,2])# consider the face with max confidence score
        confidence = detections[0,0,i,2]
        if confidence > 0.5:
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startx,starty,endx,endy) = box.astype('int')
            # step-2: Feature Extraction or Embedding
            roi = image[starty:endy,startx:endx].copy()
            # get the face descriptors
            faceblob = cv2.dnn.blobFromImage(roi,1/255,(96,96),(0,0,0),swapRB=True,crop=True)
            descriptor_model.setInput(faceblob)
            vectors = descriptor_model.forward()
            
            return vectors
    return None


In [5]:
#consider sample image

img = cv2.imread(r"C:\Users\abidh\Downloads\group photo.png")
cv2.imshow('sample',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# apply helper function to all images and get face descriptors

In [5]:
data = dict(data=[],label=[])

In [10]:
folders = os.listdir('MCET_STUDENTS')
for folder in folders:
    filenames = os.listdir('MCET_STUDENTS/{}'.format(folder))
    for filename in filenames:
        try:

            vector = helper('./MCET_STUDENTS/{}/{}'.format(folder,filename))
            if vector is not None:
                data['data'].append(vector)
                data['label'].append(folder)
                print('Feature Extracted Sucessfully')
                
        except:
            pass

Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Ex

Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Ex

In [11]:
data.keys()

dict_keys(['data', 'label'])

In [12]:
pd.Series(data['label']).value_counts()

Prattay              152
Usuf                 124
Abid                  94
Vashkar               86
Aninda Bakshi Sir     60
Tanmoy Bera Sir       48
Amit Sir              36
dtype: int64

In [13]:
# save the data
pickle.dump(data,open('data_face_features_mcet.pickle',mode='wb'))